<a href="https://colab.research.google.com/github/amodsgit/AmodTheCoder/blob/main/Falcon7b_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-Tuning the Falcon7-b LLM for Specialized Mental Health Therapy Applications using QLoRA**

A Project by Amod

Get comparable performance to full finetuning by adapting LLMs to downstream tasks using consumer hardware

Description of newer labraries used within the project:

Bitsandbytes: is a lightweight wrapper around CUDA custom functions, in particular 8-bit optimizers and quantization functions.

🤗 (Hugging Face) Accelerate: was created for PyTorch users who like to write the training loop of PyTorch models but are reluctant to write and maintain the boilerplate code needed to use multi-GPUs/TPU/fp16.


> 🤗 Accelerate abstracts exactly and only the boilerplate code related to multi-GPUs/TPU/fp16 and leaves the rest of your code unchanged.

Einops : Enables easy tensor operations

Loralib: PyTorch implementation of low-rank adaptation (LoRA), a parameter-efficient approach to adapt a large pre-trained deep learning model which obtains performance on-par with full model fine-tuning

XFormers (optional): A collection of composable Transformer building blocks.

In [ ]:
!pip install -qU bitsandbytes transformers datasets accelerate loralib einops xformers

In [ ]:
!pip install -q -U git+https://github.com/huggingface/peft.git

In [ ]:
import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
"""
please add an explanation
"""

model_id = "tiiuae/falcon-7b" # we are using falcon 7b as our base model

# bitsandbytes configuration for loading the model in NF4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model =AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model) # initialize model for QLoRA

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

"""
Meaning of the parameters in loraconfig (Include these in the code explanation) :
r: the rank of the update matrices, expressed in int. Lower rank results in smaller update matrices with fewer trainable parameters.
target_modules: The modules (for example, attention blocks) to apply the LoRA update matrices.
lora_alpha: LoRA scaling factor
bias: Specifies if the bias parameters should be trained. Can be 'none', 'all' or 'lora_only'
lora_dropout: dropout probability of the LoRA layers
"""

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


Formatting each Datapoint (Question answer pair of the dataset) to be loaded into training





In [ ]:
"""
Fuctions to Format and tokenize each Datapoint (Question answer pair of the dataset) to be loaded into training
 """

def generate_prompt(data_point):
  return f"""
<Human>: {data_point["Context"]}
<AI>: {data_point["Response"]}
  """.strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt


Loading Dataset

In [ ]:
from datasets import load_dataset
dataset_name = 'Amod/mental_health_counseling_conversations'  # a custom dataset of 3512 high quality therapy conversations
dataset = load_dataset(dataset_name, split="train")

Sampling the Dataset

In [ ]:
dataset[400]

{'Context': "When I'm around people, I sometimes think someone has made a comment to me or asked me to do something. I will be focusing on something else and then randomly think I hear something about me. I never know if it was actually said.",
 'Response': "It sounds like you are having difficulty knowing if people are saying things to you, or if you only thought someone was talking to you, and this is anxiety producing. It's understandable that it could be confusing and/or a bit scary to be unsure if people are speaking to you or not. It's hard to say exactly what is going on from just this description, but if you can work with a competent therapist, you may be able to get more insight into what is happening, get clarity about your social interactions, and develop some ways to deal with the anxiety."}

Finalizing and Mapping the dataset for finetuning

In [ ]:
"""plese add an explanation for this code: """
dataset = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [ ]:
print(dataset.shape)

(3512, 5)


In [ ]:
OUTPUT_DIR = "experiments"

In [ ]:
"""
Explain the transformers Trainer. explain each traing argument
"""

training_args = transformers.TrainingArguments(
    auto_find_batch_size=True,
    num_train_epochs=4,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=4,
    logging_steps=10,
    output_dir=OUTPUT_DIR,
    save_strategy='epoch',
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()    # Commence training
